In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib as plt
import glob
import numpy as np
from PIL import Image
import mltools as ml

trying to use the example from 
[http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)

denote number of data  
mesa: 10015  
forest: 10047   
desert: 10142  
training size: 21142  
test size: 9062  
total size:  30204

In [3]:
labels ={"mesa":0, "forest":1, "desert":2}

In [4]:
mesa_files = glob.glob( '/notebooks/Minecraft-AI/mc-data/mesa_splited/**/*.jpg')
forest_files = glob.glob('/notebooks/Minecraft-AI/mc-data/forest_splited/**/*.jpg')
desert_files = glob.glob('/notebooks/Minecraft-AI/mc-data/desert_splited/**/*.jpg')
mesa_pairs = zip(mesa_files, "0"*len(mesa_files))
forest_pairs = zip(forest_files, "1"*len(forest_files))
desert_pairs = zip(desert_files, "2"*len(desert_files))

In [5]:
print "mesa size: ", len(mesa_pairs)
print "forest size: ", len(forest_pairs)
print "desert size: ", len(desert_pairs)

mesa size:  10015
forest size:  10047
desert size:  10142


In [6]:
filename_pairs = mesa_pairs+forest_pairs+desert_pairs

In [7]:
np.random.shuffle(filename_pairs)
portion=int(0.7*len(filename_pairs))
train_pairs = filename_pairs[:portion]
test_pairs = filename_pairs[portion:]

In [8]:
print train_pairs[:5]

[('/notebooks/Minecraft-AI/mc-data/forest_splited/008/forest_7448.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/mesa_splited/006/mesa_6246.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/desert_splited/003/desert_3434.jpg', '2'), ('/notebooks/Minecraft-AI/mc-data/desert_splited/008/desert_7747.jpg', '2'), ('/notebooks/Minecraft-AI/mc-data/mesa_splited/001/mesa_1129.jpg', '0')]


In [10]:
print "training size:", len(train_pairs)
print "test size:", len(test_pairs)
print "total size: ", len(filename_pairs)

 training size: 21142
test size: 9062
total size:  30204


In [11]:
tfrecords_filename = '/notebooks/Minecraft-AI/mc-data/mesa_forest_desert_train.tfrecords'
tfrecords_test_filename='/notebooks/Minecraft-AI/mc-data/mesa_forest_desert_test.tfrecords'
train_writer = tf.python_io.TFRecordWriter(tfrecords_filename)
test_writer = tf.python_io.TFRecordWriter(tfrecords_test_filename)

In [12]:
# helper functions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [13]:
# Let's collect the real images to later on compare
# to the reconstructed ones
# original_images = []

In [14]:
def write_in_tfrecord(filename_pairs, writer):
    for img_path, label in filename_pairs:

        img = np.array(Image.open(img_path))
        
        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        # convert to 1d array and convert to respective
        # shape that image used to have.
        height = img.shape[0]
        width = img.shape[1]

        # Put in the original images into array
        # Just for future check for correctness
#         original_images.append((img,label))

        img_raw = img.tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'image_raw': _bytes_feature(img_raw),
            'label': _int64_feature(int(label))}))

        writer.write(example.SerializeToString())

In [15]:
write_in_tfrecord(train_pairs,train_writer)

In [16]:
write_in_tfrecord(test_pairs,test_writer)

In [17]:
train_writer.close()
test_writer.close()